# Showdown Card Creation Script

In [16]:
from mlb_showdown_bot.baseball_ref_scraper import BaseballReferenceScraper
from mlb_showdown_bot.showdown_player_card_generator import ShowdownPlayerCardGenerator
import os
import pandas as pd
import numpy as np
import datetime as dt
from PIL import Image, ImageDraw

## Determining Players to Create Cards For

We won't be just picking every player to create a card for since a.) it would take forever for me to get images for each player every season and b.) it'll take up a ton of space. What we'll end up doing is going season by season and picking top performers we want to create cards for manually.

We'll pull data from baseball reference and look at the top WAR players by position for that season. We'll create a function to pull `x` top performers at a given position for a given season by WAR.

I'm currently manually pulling csv's from baseball reference. Might be good in the future to scrape it directly via python to remove a manual step.

The following function will take in a given `season`, `position` and `top_n` value and return the top n players by total WAR at that position for that season. Couple of things to note:

* Position is going to be based off of where the user played the majority of their games
* WAR is summed up across pitching and hitting
* Pitchers are called starters if they started at least `50%` of their games, relievers if they had started less than `50%`

In [157]:
def top_x(season, position, top_n):
    # Identifying directory we should pull player data from
    directory = "player_data/" + str(season) + "/"
    
    # Reading in player data
    position_player_war = pd.read_csv(directory + "position_player_war.csv")
    position_player_war["war_type"] = "positon"
    pitcher_war = pd.read_csv(directory + "pitcher_war.csv")
    pitcher_war["war_type"] = "pitcher"
    position_games = pd.read_csv(directory + "position.csv")

    # Binding position player and pitcher WAR data
    all_players_war = pd.concat([position_player_war, pitcher_war])
    # Removing any rows with NA WAR values
    all_players_war = all_players_war.dropna(subset = "WAR")
    # Summing up pitcher and position WAR by player
    all_players_war = all_players_war.groupby("Name-additional")["WAR"].sum().reset_index()

    # Now joining position data with total war data
    all_players_war_position = pd.merge(all_players_war, position_games, on = ["Name-additional"])
    # Putting position game values from wide to long
    all_players_war_position = all_players_war_position.melt(id_vars = ["Name", "WAR", "Name-additional", "G", "GS"])
    # Getting position that player had the most games at
    # Note that a player might have duplicate values if they played the same amount of games at multiple positions
    all_players_war_position = all_players_war_position.loc[all_players_war_position.groupby("Name-additional")["value"].idxmax()]
    # To determine SP vs RP, we'll find rate of games started
    all_players_war_position["GS_rate"] = all_players_war_position["GS"] / all_players_war_position["G"]
    all_players_war_position["variable"] = np.where((all_players_war_position["variable"] == "P") & (all_players_war_position["GS_rate"] >= .5), "SP",
        np.where((all_players_war_position["variable"] == "P") & (all_players_war_position["GS_rate"] < .5), "RP", all_players_war_position["variable"]))

    # We'll now filter for the given position the function specifies and finding top n
    final = all_players_war_position[all_players_war_position["variable"] == position].nlargest(top_n, "WAR")
    # Cleaning up data frame
    final = final[["Name", "Name-additional", "WAR"]].sort_values("WAR", ascending = False)
    
    return(final)


In [169]:
top_x(2010, "DH", 10)

,Name,Name-additional,WAR
13523,Luke Scott,scottlu01,3.8
13614,Jim Thome,thomeji01,3.6
12751,Johnny Damon,damonjo01,3.0
13323,David Ortiz,ortizda01,2.8
12926,Travis Hafner,hafnetr01,2.4
12748,Jack Cust,custja01,2.3
12614,Russell Branyan,branyru01,2.0
12917,Vladimir Guerrero,guerrvl01,1.8
13193,Hideki Matsui,matsuhi01,1.7
13588,Mike Sweeney,sweenmi01,0.6


Once we manually go through our lists, we manually grab a player image from that season and put it in the [raw images](card_images/raw_images) directory for a given season. The file should be named `FirstName_LastName-Season`.

## Generating Card Images

After we add player images to the raw images folder, we'll start generating cards. We essentially use the raw images directory to identify which players should have cards generated.

The following function takes in a `season` input and will generate cards from images within that directory. The general process is:

* Read through all images in that directory and create a tuple that stores the player's name, the season, and the file name (both with and without the extension)
* We then check to see if the player already has a card generated in the output [card images](card_images/card_images/) directory. If they do, we won't create another card for them
* Any player that does not have a card generated already will have a new card generated
* The returned output of the function is a dataframe of the charts for these newly added players

In [20]:
def card_generator(season):
    # Getting raw image directory
    raw_image_directory = "card_images/raw_images/" + str(season) + "/"
    card_image_directory = "showdown_game/www/card_images/" + str(season) + "/"
    
    # Looping through each raw image in given directory and creating player tuple
    value_list = []
    for i in os.listdir(raw_image_directory):
        split_value = i.split("-")
        
        name_value = split_value[0].replace("_", " ")

        year_value = split_value[1].split(".")[0]
        
        value_tuple = (name_value, year_value, name_value.replace(" ", "_") + "-" + year_value, i)
        value_list.append(value_tuple)

    # Now identifying what cards exist already and filtering them out of list to create
    raw_images = [os.path.splitext(i)[0] for i in os.listdir(raw_image_directory)]
    card_images = [os.path.splitext(i)[0] for i in os.listdir(card_image_directory)]
    cards_to_create = [i for i in raw_images if i not in card_images]
    value_list_filtered = [i for i in value_list if i[2] in cards_to_create]

    # Now generating card and charts using showdown library
    df_list = list()
    for i in value_list_filtered:
        scraper = BaseballReferenceScraper(name = i[0], year = i[1])
        statline = scraper.player_statline()

        showdown = ShowdownPlayerCardGenerator(
            name = i[0],
            year = i[1],
            stats = statline,
            context = "2022-CLASSIC",
            print_to_cli = True,
            card_img_output_folder_path = card_image_directory,
            player_image_path = os.getcwd() + "/" + raw_image_directory + i[3])

        card_path = card_image_directory + showdown.image_name

        image = Image.open(card_path)
        radius = 60
        circle = Image.new ('L', (radius * 2, radius * 2), 0)
        draw = ImageDraw.Draw (circle)
        draw.ellipse ((0, 0, radius * 2, radius * 2), fill = 255)
        alpha = Image.new ('L', image.size, 255)
        w, h = image.size
        alpha.paste (circle.crop ((0, 0, radius, radius)), (0, 0))
        alpha.paste (circle.crop ((0, radius, radius, radius * 2)), (0, h-radius))
        alpha.paste (circle.crop ((radius, 0, radius * 2, radius)), (w-radius, 0))
        alpha.paste (circle.crop ((radius, radius, radius * 2, radius * 2)), (w-radius, h-radius))
        image.putalpha (alpha)
        image.save(card_path)

        old_ext = os.path.splitext(showdown.image_name)[1]
        os.rename(card_path, card_image_directory + i[2] + old_ext)

        chart_df = pd.DataFrame(showdown.chart_ranges, index = [0])

        chart_df = chart_df.rename(columns = {"so Range": "so_range", "gb Range": "gb_range", "fb Range": "fb_range",
                                            "bb Range": "bb_range", "1b Range": "single_range", "1b+ Range": "single_plus_range",
                                            "2b Range": "double_range", "3b Range": "triple_range", "hr Range": "hr_range"})

        player_df = pd.DataFrame({"player": showdown.name,
                                "year": showdown.year,
                                "points": showdown.points,
                                "command": showdown.chart["command"],
                                "main_position": list(showdown.positions_and_defense.keys())[0],
                                "card_path": card_image_directory + i[2] + old_ext}, index = [0])

        combo_df = pd.concat([player_df, chart_df], axis = 1)

        df_list.append(combo_df)

    final_df = pd.concat(df_list)
    final_df["ts"] = dt.datetime.now()

    return(final_df)

    

In [28]:
%%capture

new_players = card_generator(2001)

With cards generated, we have a returned output of charts for new players who had their cards generated. We should append these charts to our player db that contains charts for everyone who has a card generated.

In [29]:
player_db = pd.read_csv("data/player_db.csv")
player_db = pd.concat([player_db, new_players]).reset_index(drop = True)
player_db["ts"] = pd.to_datetime(player_db["ts"])
player_db["year"] = player_db["year"].astype("int")
player_db = player_db.loc[player_db.groupby(["player", "year"])["ts"].idxmax()].sort_values(["player", "year"])

In [30]:
os.replace("data/player_db.csv", "data/player_db_backup.csv")
player_db.to_csv("data/player_db.csv", index = False)
pd.read_csv("data/player_db.csv").head()

,player,year,points,command,main_position,card_path,so_range,gb_range,fb_range,bb_range,single_range,single_plus_range,double_range,triple_range,hr_range,pu Range,ts
0,Aaron Judge,2022,700,10,OF,showdown_game/www/card_images/2022/aaron_judge...,1–2,—,—,3–9,10–13,14,15,—,16–20,NaN,2024-02-18 11:16:21.823015
1,Aaron Nola,2022,510,3,STARTER,showdown_game/www/card_images/2022/aaron_nola-...,4–10,11–15,16–18,—,19,NaN,20,NaN,—,1–3,2024-02-18 11:16:21.823015
2,Adrián Beltré,2010,470,9,3B,showdown_game/www/card_images/2010/adrian_belt...,—,1–2,3–4,5–6,7–13,—,14–17,—,18–20,NaN,2024-02-18 11:27:30.941819
3,Albert Pujols,2010,550,10,1B,showdown_game/www/card_images/2010/albert_pujo...,—,1,2–3,4–9,10–14,15,16–17,—,18–20,NaN,2024-02-18 11:27:30.941819
4,Alek Manoah,2022,540,4,STARTER,showdown_game/www/card_images/2022/alek_manoah...,4–8,9–13,14–17,18,19,NaN,20,NaN,—,1–3,2024-02-18 11:16:21.823015
